In [107]:
import pandas as pd
import numpy as np
from sklearn import neighbors
from sklearn.model_selection import train_test_split

In [111]:
df= pd.read_pickle("df2.pkl")

In [129]:
Y = df["label"]
X = df.drop("label", axis=1)

In [20]:
df.head()

,id,date,source,title,content,published_utc,word_count,label,bias,factuality,...,country_Israel,country_Japan,country_Kenya,country_Macedonia,country_Netherlands,country_Qatar,country_Russia,country_USA,country_UnitedKingdom,country_Unknown
1315620,21stcenturywire--2022-09-27--Joseph Arthur wit...,2022-09-27,21stcenturywire,Joseph Arthur with guest Patrick Henningsen on...,In this episode of the Joseph Arthur & His Tec...,1664260880,134,1.0,conspiracy-pseudoscience,2.0,...,0,0,0,0,0,0,0,1,0,0
1501555,21stcenturywire--2022-11-03--IEA: EU Gas Crisi...,2022-11-03,21stcenturywire,IEA: EU Gas Crisis May Worsen in 2023,The EU’s energy crisis may last through to 202...,1667527821,564,1.0,conspiracy-pseudoscience,2.0,...,0,0,0,0,0,0,0,1,0,0
1482658,21stcenturywire--2022-10-31--US Supreme Court ...,2022-10-31,21stcenturywire,US Supreme Court May End ‘Positive Discriminat...,Conservative Supreme Court justices on Monday ...,1667262392,414,1.0,conspiracy-pseudoscience,2.0,...,0,0,0,0,0,0,0,1,0,0
1664306,21stcenturywire--2022-12-07--When ‘Children of...,2022-12-07,21stcenturywire,When ‘Children of Men’ Gets Real: Infertility ...,Children of Men was a dystopian 1992 science f...,1670470004,1244,1.0,conspiracy-pseudoscience,2.0,...,0,0,0,0,0,0,0,1,0,0
1630475,21stcenturywire--2022-11-30--CBDC in Beta Mode...,2022-11-30,21stcenturywire,CBDC in Beta Mode: Get Ready for ‘Fed Coin’,While Americans were preparing for Thanksgivin...,1669834537,233,1.0,conspiracy-pseudoscience,2.0,...,0,0,0,0,0,0,0,1,0,0


In [130]:
X.columns = X.columns.astype(str)

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85051 entries, 1315620 to 753324
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        85051 non-null  object 
 1   date                      85051 non-null  object 
 2   source                    85051 non-null  object 
 3   title                     85051 non-null  object 
 4   content                   85051 non-null  object 
 5   published_utc             85051 non-null  int64  
 6   word_count                85051 non-null  int64  
 7   bias                      85051 non-null  object 
 8   factuality                85051 non-null  float64
 9   questionable-source       85051 non-null  float64
 10  conspiracy-pseudoscience  85051 non-null  float64
 11  pro-science               85051 non-null  float64
 12  year                      85051 non-null  int64  
 13  month                     85051 non-null  int64  
 14 

In [114]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [212]:
from scipy.sparse import hstack

# Create two CountVectorizer objects
vectorizer1 = CountVectorizer()
vectorizer2 = CountVectorizer()

# Fit the vectorizers on the training data
vectorizer1.fit(X_train['content'])
vectorizer2.fit(X_train['title'])

# Transform the training and test data
X_train_content = vectorizer1.transform(X_train['content'])
X_train_title = vectorizer2.transform(X_train['title'])
X_test_content = vectorizer1.transform(X_test['content'])
X_test_title = vectorizer2.transform(X_test['title'])

# Combine the feature matrices
X_train_text = hstack([X_train_content, X_train_title])
X_test_text = hstack([X_test_content, X_test_title])

In [43]:
knn = neighbors.KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_text, y_train)

KNeighborsClassifier(n_neighbors=3)

In [45]:
knn.score(X_test_text, y_test)

0.5236023749338663

In [48]:
y_pred = knn.predict(X_test_text)
pd.crosstab(y_test, y_pred, rownames=["real"], colnames=["predict"])

predict,0.0,1.0,2.0
real,,,
0.0,5359,1122,798
1.0,2754,2256,389
2.0,2487,554,1292


In [49]:
from scipy.sparse import hstack, csr_matrix

# Select the numeric and uint8 columns
X_train_numeric = X_train.select_dtypes(include=['int64', 'float64', 'uint8'])
X_test_numeric = X_test.select_dtypes(include=['int64', 'float64', 'uint8'])

# Convert the selected columns to a sparse matrix format
X_train_numeric = csr_matrix(X_train_numeric.values)
X_test_numeric = csr_matrix(X_test_numeric.values)

# Combine the feature matrices
X_train = hstack([X_train_content, X_train_title, X_train_numeric])
X_test = hstack([X_test_content, X_test_title, X_test_numeric])

In [55]:
knn = neighbors.KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [56]:
knn.score(X_test, y_test)

0.43565927928987125

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
vectorizer1 = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()

vectorizer1.fit(X_train['content'])
vectorizer2.fit(X_train['title'])

X_train_content = vectorizer1.transform(X_train['content'])
X_train_title = vectorizer2.transform(X_train['title'])
X_test_content = vectorizer1.transform(X_test['content'])
X_test_title = vectorizer2.transform(X_test['title'])

X_train_numeric = X_train.select_dtypes(include=['int64', 'float64', 'uint8'])
X_test_numeric = X_test.select_dtypes(include=['int64', 'float64', 'uint8'])

X_train_numeric = csr_matrix(X_train_numeric.values)
X_test_numeric = csr_matrix(X_test_numeric.values)

X_train = hstack([X_train_content, X_train_title, X_train_numeric])
X_test = hstack([X_test_content, X_test_title, X_test_numeric])

In [22]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_text, y_train)

MultinomialNB()

In [23]:
clf.score(X_test_text, y_test)

0.6138381047557463

In [24]:
y_pred = clf.predict(X_test_text)
pd.crosstab(y_test, y_pred, rownames=["real"], colnames=["predict"])

predict,0.0,1.0,2.0
real,,,
0.0,4248,1428,1678
1.0,831,3658,829
2.0,1150,653,2536


In [11]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [12]:
clf.score(X_test, y_test)

0.3991534889189348

In [17]:
vectorizer1 = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()

vectorizer1.fit(X_train['content'])
vectorizer2.fit(X_train['title'])

X_train_content = vectorizer1.transform(X_train['content'])
X_train_title = vectorizer2.transform(X_train['title'])
X_test_content = vectorizer1.transform(X_test['content'])
X_test_title = vectorizer2.transform(X_test['title'])

X_train_text = hstack([X_train_content, X_train_title])
X_test_text = hstack([X_test_content, X_test_title])

In [100]:
clf = MultinomialNB(alpha=0.0015)
clf.fit(X_train_text, y_train)

MultinomialNB(alpha=0.0015)

In [101]:
clf.score(X_test_text, y_test)

0.6191875845041444

In [102]:
y_pred = clf.predict(X_test_text)
pd.crosstab(y_test, y_pred, rownames=["real"], colnames=["predict"])

predict,0.0,1.0,2.0
real,,,
0.0,4348,1374,1632
1.0,915,3632,771
2.0,1142,644,2553


In [235]:
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB(alpha=0.0015)
clf.fit(X_train_text, y_train)

ComplementNB(alpha=0.0015)

In [236]:
clf.score(X_test_text, y_test)

0.6233613544177297

In [232]:
y_pred = clf.predict(X_test_text)
pd.crosstab(y_test, y_pred, rownames=["real"], colnames=["predict"])

predict,0.0,1.0,2.0
real,,,
0.0,4318,1506,1461
1.0,736,3823,727
2.0,1187,790,2463


In [103]:
vectorizer1 = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()

vectorizer1.fit(X_train['content'])
vectorizer2.fit(X_train['title'])

X_train_content = vectorizer1.transform(X_train['content'])
X_train_title = vectorizer2.transform(X_train['title'])
X_test_content = vectorizer1.transform(X_test['content'])
X_test_title = vectorizer2.transform(X_test['title'])

X_train_text = hstack([X_train_content, X_train_title])
X_test_text = hstack([X_test_content, X_test_title])

In [104]:
clf = ComplementNB(alpha=0.0015)
clf.fit(X_train_text, y_train)

ComplementNB(alpha=0.0015)

In [105]:
clf.score(X_test_text, y_test)

0.6146023161483746

In [106]:
y_pred = clf.predict(X_test_text)
pd.crosstab(y_test, y_pred, rownames=["real"], colnames=["predict"])

predict,0.0,1.0,2.0
real,,,
0.0,4357,1537,1460
1.0,894,3760,664
2.0,1209,792,2338


In [109]:
from sklearn.naive_bayes import CategoricalNB

In [171]:
X_train_numeric = X_train.select_dtypes(include=['int64', 'float64', 'uint8'])
X_test_numeric = X_test.select_dtypes(include=['int64', 'float64', 'uint8'])
#X_train_numeric = csr_matrix(X_train_numeric.values)
#X_test_numeric = csr_matrix(X_test_numeric.values)
#X_train_numeric = X_train_numeric.toarray()
#X_test_numeric = X_test_numeric.toarray()

In [173]:
X_train_numeric["word_count"] = X_train_numeric["word_count"].astype('int32')

In [175]:
int64_columns = X_train_numeric.select_dtypes(include=['int64']).columns
X_train_numeric[int64_columns] = X_train_numeric[int64_columns].astype('int16')
X_train_numeric = X_train_numeric.drop("published_utc", axis=1)

In [176]:
float64_columns = X_train_numeric.select_dtypes(include=["float64"]).columns
X_train_numeric[float64_columns] = X_train_numeric[float64_columns].astype("int8")

In [177]:
X_test_numeric["word_count"] = X_test_numeric["word_count"].astype('int32')
int64_columns = X_test_numeric.select_dtypes(include=['int64']).columns
X_test_numeric[int64_columns] = X_test_numeric[int64_columns].astype('int16')
X_test_numeric = X_test_numeric.drop("published_utc", axis=1)
float64_columns = X_test_numeric.select_dtypes(include=["float64"]).columns
X_test_numeric[float64_columns] = X_test_numeric[float64_columns].astype("int8")

In [178]:
print(X_train_numeric.shape)
print(X_train_numeric.dtypes)

(68040, 27)
word_count                  int32
factuality                   int8
questionable-source          int8
conspiracy-pseudoscience     int8
pro-science                  int8
year                        int16
month                       int16
dayofyear                   int16
dayofthemonth               int16
weekday                     int16
country_Australia           uint8
country_Bulgaria            uint8
country_Canada              uint8
country_Cyprus              uint8
country_France              uint8
country_Germany             uint8
country_Ireland             uint8
country_Israel              uint8
country_Japan               uint8
country_Kenya               uint8
country_Macedonia           uint8
country_Netherlands         uint8
country_Qatar               uint8
country_Russia              uint8
country_USA                 uint8
country_UnitedKingdom       uint8
country_Unknown             uint8
dtype: object


In [169]:
(X_test_numeric < 0).any()

word_count                   True
factuality                  False
questionable-source         False
conspiracy-pseudoscience    False
pro-science                 False
year                        False
month                       False
dayofyear                   False
dayofthemonth               False
weekday                     False
country_Australia           False
country_Bulgaria            False
country_Canada              False
country_Cyprus              False
country_France              False
country_Germany             False
country_Ireland             False
country_Israel              False
country_Japan               False
country_Kenya               False
country_Macedonia           False
country_Netherlands         False
country_Qatar               False
country_Russia              False
country_USA                 False
country_UnitedKingdom       False
country_Unknown             False
dtype: bool

In [179]:
X_test_numeric[(X_test_numeric < 0).any(axis=1)].head()

,word_count,factuality,questionable-source,conspiracy-pseudoscience,pro-science,year,month,dayofyear,dayofthemonth,weekday,...,country_Israel,country_Japan,country_Kenya,country_Macedonia,country_Netherlands,country_Qatar,country_Russia,country_USA,country_UnitedKingdom,country_Unknown


In [237]:
cat_nb = CategoricalNB()
cat_nb.fit(X_train_numeric, y_train)

multi_nb = MultinomialNB(alpha=0.0015)
multi_nb.fit(X_train_text, y_train)

cat_prob = cat_nb.predict_proba(X_test_numeric)
multi_prob = multi_nb.predict_proba(X_test_text)


In [238]:
avg_prob = (cat_prob + multi_prob) / 2

In [239]:
cat_nb.score(X_test_numeric, y_test)

0.9225207218858386

In [240]:
multi_nb.score(X_test_text, y_test)

0.6146611016401152

In [187]:
#final_prob_test = final_prob*cat_nb.class_prior_

In [195]:
from sklearn.metrics import accuracy_score

In [241]:
y_pred = np.argmax(avg_prob, axis=1)

accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.7318205866792076


In [242]:
pd.crosstab(y_test, y_pred, rownames=["real"], colnames=["predict"])

predict,0,1,2
real,,,
0.0,5284,799,1202
1.0,397,4431,458
2.0,1093,613,2734
